In [1]:
import boto3

bucket_list_dataStationInformation = []

s3 = boto3.resource('s3')
s3_object = s3.Bucket('citibikejsonfiles')

for file in s3_object.objects.filter(Prefix = 'dataStationInformation'):
    file_name = file.key
    if file_name.find(".json") != -1:
        bucket_list_dataStationInformation.append(file.key)

len_bucket_list_dataStationInformation = print(len(bucket_list_dataStationInformation))
print(bucket_list_dataStationInformation[0:10])


998
['dataStationInformation 2023-01-14 01:40:37.364932.json', 'dataStationInformation 2023-01-14 01:50:37.351757.json', 'dataStationInformation 2023-01-14 02:00:37.354085.json', 'dataStationInformation 2023-01-14 02:10:37.297757.json', 'dataStationInformation 2023-01-14 02:20:37.491889.json', 'dataStationInformation 2023-01-14 02:30:37.376058.json', 'dataStationInformation 2023-01-14 02:40:37.058998.json', 'dataStationInformation 2023-01-14 02:50:37.666289.json', 'dataStationInformation 2023-01-14 03:00:37.288828.json', 'dataStationInformation 2023-01-14 03:10:37.558505.json']


In [2]:
# To import dataStationStatus s3 files

bucket_list_dataStationStatus = []

for file in s3_object.objects.filter(Prefix = 'dataStationStatus'):
    file_name_status = file.key
    if file_name_status.find(".json") != -1:
        bucket_list_dataStationStatus.append(file.key)

len_bucket_list_dataStationStatus = print(len(bucket_list_dataStationStatus))
print(bucket_list_dataStationStatus[0:10])

998
['dataStationStatus 2023-01-14 01:40:36.938626.json', 'dataStationStatus 2023-01-14 01:50:36.909127.json', 'dataStationStatus 2023-01-14 02:00:36.817121.json', 'dataStationStatus 2023-01-14 02:10:36.800837.json', 'dataStationStatus 2023-01-14 02:20:37.034204.json', 'dataStationStatus 2023-01-14 02:30:36.863208.json', 'dataStationStatus 2023-01-14 02:40:36.607410.json', 'dataStationStatus 2023-01-14 02:50:37.224251.json', 'dataStationStatus 2023-01-14 03:00:36.820247.json', 'dataStationStatus 2023-01-14 03:10:37.096055.json']


In [3]:
# To get the timestamp key for DataStationInformation
dataframe_key_dataStationInformation = []

for file in bucket_list_dataStationInformation:
    file_name = file[23:39]
    dataframe_key_dataStationInformation.append(file_name)

dataframe_key_dataStationInformation[:5]

['2023-01-14 01:40',
 '2023-01-14 01:50',
 '2023-01-14 02:00',
 '2023-01-14 02:10',
 '2023-01-14 02:20']

In [4]:
# To get the timestamp key for DataStationStatus
dataframe_key_dataStationStatus = []

for file in bucket_list_dataStationStatus:
    file_name = file[-31:-15] # need to update
    dataframe_key_dataStationStatus.append(file_name)

dataframe_key_dataStationStatus[:5]

['2023-01-14 01:40',
 '2023-01-14 01:50',
 '2023-01-14 02:00',
 '2023-01-14 02:10',
 '2023-01-14 02:20']

In [5]:
from io import StringIO
import io
import pandas as pd

dataframe_dataStationInformation = [] # empty list of dataframes

for file in bucket_list_dataStationInformation:
    s3_obj = s3.Object('citibikejsonfiles', file)
    data = s3_obj.get()['Body'].read()
    dataframe_dataStationInformation.append(pd.read_json(io.BytesIO(data)))

,data,last_updated,ttl
stations,"[{'eightd_has_key_dispenser': False, 'short_na...",1673660428,5


In [8]:
# StationInformationTimeSeries is the DataFrame

appended_data = []
for key in range(len(dataframe_dataStationInformation)):
    tmpDataStationInformation = pd.json_normalize(dataframe_dataStationInformation[key]['data']['stations'])
    tmpDataStationInformation['time_stamp'] = dataframe_key_dataStationInformation[key]
    appended_data.append(tmpDataStationInformation)
StationInformationTimeSeries = pd.concat(appended_data, ignore_index = True)

# station_id, name, short_name*, lat, lon, time_stamp, capacity, region_id

In [14]:
# Need to investigate NaN to see where we can group these entries

#  region_id NaN only belongs to station_id 4987 and it can be dropped from
# the entire dataframe
StationInformationTimeSeries["region_id"].unique()


array(['70', '71', '311', nan], dtype=object)

In [16]:
# Station ID 4987 may be out of order or removed and can be purged from the list, it is safe to assume it is just being logged and someone forgot to remove it from the real time feed
StationInformationTimeSeries[StationInformationTimeSeries.region_id.isnull()]

,eightd_has_key_dispenser,short_name,capacity,rental_methods,name,lon,eightd_station_services,station_type,electric_bike_surcharge_waiver,has_kiosk,lat,legacy_id,external_id,region_id,station_id,rental_uris.ios,rental_uris.android,time_stamp
1807,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
3641,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:50
5475,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:00
7309,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:10
9143,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824255,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:10
1826100,False,4567.07,0,"[KEY, CREDITCARD]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:20
1827945,False,4567.07,0,"[KEY, CREDITCARD]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:30
1829790,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:40


In [17]:
StationInformationTimeSeries[(StationInformationTimeSeries.lat == 0.0) & (StationInformationTimeSeries.lon == 0.0)]

,eightd_has_key_dispenser,short_name,capacity,rental_methods,name,lon,eightd_station_services,station_type,electric_bike_surcharge_waiver,has_kiosk,lat,legacy_id,external_id,region_id,station_id,rental_uris.ios,rental_uris.android,time_stamp
1807,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
3641,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:50
5475,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:00
7309,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:10
9143,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 02:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824255,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:10
1826100,False,4567.07,0,"[KEY, CREDITCARD]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:20
1827945,False,4567.07,0,"[KEY, CREDITCARD]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:30
1829790,False,4567.07,0,"[CREDITCARD, KEY]",4567.07,0.0,[],classic,False,False,0.0,4987,a3936c86-4551-4c08-9fe4-ddd87a727205,NaN,4987,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-20 23:40


In [12]:
type(StationInformationTimeSeries)

pandas.core.frame.DataFrame

In [9]:
StationInformationTimeSeries.head()

,eightd_has_key_dispenser,short_name,capacity,rental_methods,name,lon,eightd_station_services,station_type,electric_bike_surcharge_waiver,has_kiosk,lat,legacy_id,external_id,region_id,station_id,rental_uris.ios,rental_uris.android,time_stamp
0,False,JC003,22,"[KEY, CREDITCARD]",City Hall,-74.043845,[],classic,False,True,40.717732,3185,66dd5960-0aca-11e7-82f6-3863bb44ef7c,70,66dd5960-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
1,False,JC002,14,"[KEY, CREDITCARD]",Paulus Hook,-74.033552,[],classic,False,True,40.714145,3184,66dd58f2-0aca-11e7-82f6-3863bb44ef7c,70,66dd58f2-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
2,False,JC008,14,"[KEY, CREDITCARD]",Newport Pkwy,-74.032108,[],classic,False,True,40.728745,3199,66dd6441-0aca-11e7-82f6-3863bb44ef7c,70,66dd6441-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
3,False,JC009,26,"[KEY, CREDITCARD]",Hamilton Park,-74.044247,[],classic,False,True,40.727596,3203,66dd7b37-0aca-11e7-82f6-3863bb44ef7c,70,66dd7b37-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
4,False,JC006,22,"[KEY, CREDITCARD]",Warren St,-74.038051,[],classic,False,True,40.721124,3187,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,70,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40


In [10]:
StationInformationTimeSeries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1831671 entries, 0 to 1831670
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   eightd_has_key_dispenser        bool   
 1   short_name                      object 
 2   capacity                        int64  
 3   rental_methods                  object 
 4   name                            object 
 5   lon                             float64
 6   eightd_station_services         object 
 7   station_type                    object 
 8   electric_bike_surcharge_waiver  bool   
 9   has_kiosk                       bool   
 10  lat                             float64
 11  legacy_id                       object 
 12  external_id                     object 
 13  region_id                       object 
 14  station_id                      object 
 15  rental_uris.ios                 object 
 16  rental_uris.android             object 
 17  time_stamp                 